In [1]:
import pandas as pd
import os

In [2]:
def df_only_pmsp(path_csvs):
    
    csvs = [os.path.join(path_csvs, file) for file in os.listdir(path_csvs)]
    dfs =[]

    for csv in csvs:
        try:
            df = pd.read_csv(csv, sep = ';', encoding = 'Latin-1')
            #filtra para muncipio sp:
            df = df[df['CODMUNOCOR'].apply(str).str.contains('355030')].copy().reset_index(drop = True) 
            dfs.append(df)
        except Exception as e:
            print(f'Erro ao ler o arquivo {csv}: {repr(e)}')

    df = pd.concat(dfs)
    
    return df

In [3]:
def arrumar_dt_obito(df):
    
    def tratar_data(item):
        item = str(item)
        if len(item)==7:
            dia = '0' + item[0]
            mes = item[1:3]
            ano = item[3:]
        else:
            dia = item[:2]
            mes = item[2:4]
            ano = item[4:]

        return f'{dia}/{mes}/{ano}'
    
    df['DTOBITO_Clean'] = df['DTOBITO'].apply(tratar_data)

In [4]:
def traduzir_loc_ocorr(df):
    
    def aux(item):
        
        mapper = {
            1 : 'hospital', 
            2 : 'outros estabelecimentos de saúde', 
            3 : 'domicílio', 
            4 : 'via pública', 
            5 : 'outros', 
            6 : 'aldeia indígena', 
            9 : 'ignorado',
        }
        
        if pd.isnull(item):
            return 'não preenchido'
        elif item == 0:
            return 'não preenchido'
        else:
            item = int(item)
        
        return mapper[item]
    
    df['LocalOcorrenciaTexto'] = df['LOCOCOR'].apply(aux)

In [5]:
path_csvs = 'SIM_SP_CSV'

In [6]:
df = df_only_pmsp(path_csvs)

C:\Users\h-pgy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\h-pgy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\h-pgy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\h-pgy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (17,26) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\h-pgy\Anaconda3\lib\site-packages

In [7]:
df['CODMUNOCOR'].unique()

array([3550308,  355030], dtype=int64)

In [8]:
colunas = ['DTOBITO', 'CODMUNOCOR', 'CODMUNRES', 'CAUSABAS', 'LOCOCOR']

In [9]:
df = df[colunas]

In [10]:
arrumar_dt_obito(df)

In [11]:
traduzir_loc_ocorr(df)

In [12]:
df.head()

,DTOBITO,CODMUNOCOR,CODMUNRES,CAUSABAS,LOCOCOR,DTOBITO_Clean,LocalOcorrenciaTexto
0,15021996,3550308,3550308,Y349,NaN,15/02/1996,não preenchido
1,28091996,3550308,3550308,V892,4.0,28/09/1996,via pública
2,8011996,3550308,3550308,Y349,4.0,08/01/1996,via pública
3,2011996,3550308,3534401,C187,1.0,02/01/1996,hospital
4,1011996,3550308,3550308,I694,1.0,01/01/1996,hospital


In [13]:
df['ano'] = df['DTOBITO'].apply(lambda x: str(x)[-4:])

In [14]:
df['ano'].unique()

array(['1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2008', '2009', '2013', '2014', '2015',
       '2016', '2017', '2018'], dtype=object)

In [15]:
df.to_csv('DadosObitoSampa.csv', encoding = 'latin-1', sep = ';')

In [20]:
df[df['ano']=='2018'].to_csv('DadosObitoSampa_2018.csv', encoding = 'latin-1', sep = ';')